# Implementing advanced procedures and algorithms in TensorFlow

This notebook collects advanced procedures relevant to training more complex neural networks. It gives you the possibility to look up the procedures as needed and copy the relevant code. The notebook builds on the books by Aurélien Géron and François Chollet.

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os, random

# 1. From the previous notebook: A multiclass-classification problem

Throughout, we will train a neural network on a dataset of fashion-products that is labeled with the categories of each product. The data is loaded directly from TensorFlow (which has quite the broad collection of datasets):

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_other, y_other), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
print(X_other.shape)
print(y_other.shape)
print(X_test.shape)
print(y_test.shape)

The X's are matrices (with 28x28 pixels), while the y's are numbers.
Let's plot two examples:

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(X_test[0],cmap="binary")
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(X_test[500],cmap="binary")
plt.axis('off')

As well as the corresponding labels:

In [ ]:
print(y_test[0])
print(y_test[500])

That's a bit hard to interpret. Luckily, we have the right names for each label available:

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

We can now take another look at what the pictures above represent:

In [ ]:
print(class_names[y_test[0]])
print(class_names[y_test[500]])

Finally, we divide the values of X by 255 (essentially standardizing the pixel-values to 0-1) and also split apart a validation set (of the same size as the test set):

In [ ]:
X_other = X_other / 255.
X_test = X_test / 255.
X_train, X_valid, y_train, y_valid = train_test_split(X_other, y_other, train_size = 50000, random_state=152)

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

# 2. Help! My model won't train

Sometimes models just won't train, or they only achieve very bad performance. Don't worry, luckily we can always train *some* model, because we can even learn from random data (now, whether this is desirable is another question). Let's see what could be the issue with training our model:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

Use the `summary` function, to see whether everything worked out as it should. If we defined the model as discussed above, we should get a total of 407,050 parameters.

In [ ]:
model.summary()

We can now compile the model. We use `optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=1.0)` and `metrics=['accuracy']`. For the loss, we use `sparse_categorical_crossentropy`. This is because our y's here are **not** one-hot-encoded, but instead are values from 0 to 9.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=1.0),
              metrics=["accuracy"])

Train the model for 15 epochs, keeping track also of the `validation_data`.

In [ ]:
log = model.fit(X_train, y_train, epochs=10, batch_size = 128,
                    validation_data=(X_valid, y_valid))

Take a look at the training process:

In [ ]:
def create_plot(log):
    plt.plot(log.history['accuracy'],label = "training accuracy",color='green')
    plt.plot(log.history['loss'],label = "training loss",color='darkgreen')
    plt.plot(log.history['val_accuracy'], label = "validation accuracy",color='grey')
    plt.plot(log.history['val_loss'], label = "validation loss",color='darkblue')
    plt.legend()
    plt.show()
create_plot(log)

Of course, we are not making great predictions with our model:

In [ ]:
X_new = X_test[:4]
y_predict = np.argmax(model.predict(X_new), axis=-1)
y_predict = [class_names[y] for y in y_predict]
plt.subplot(1, 4, 1)
plt.imshow(X_test[0],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 2)
plt.imshow(X_test[1],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 3)
plt.imshow(X_test[2],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 4)
plt.imshow(X_test[3],cmap="binary")
plt.axis('off')
print("Predictions are: " + str(y_predict))

Let's try this again, by lowering our learning rate:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01),
              metrics=["accuracy"])
log = model.fit(X_train, y_train, epochs=10, batch_size = 128,
                    validation_data=(X_valid, y_valid))
create_plot(log)

Looking much better, right? Our predictions also make somewhat more sense:

In [ ]:
X_new = X_test[:4]
y_predict = np.argmax(model.predict(X_new), axis=-1)
y_predict = [class_names[y] for y in y_predict]
plt.subplot(1, 4, 1)
plt.imshow(X_test[0],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 2)
plt.imshow(X_test[1],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 3)
plt.imshow(X_test[2],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 4)
plt.imshow(X_test[3],cmap="binary")
plt.axis('off')
print("Predictions are: " + str(y_predict))

## 2.1 Other changes that may be useful

### Defining the batch size

You might not have noticed, but we run mini-batch gradient descent by default. We can control the batch-size within the `model.fit` function. The default is `32`. Run the below code:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01),
              metrics=["accuracy"])
log = model.fit(X_train, y_train,
                epochs=2,batch_size=8,
                validation_data=(X_valid, y_valid))

Can you remake the model, but change the `batch_size` to `1024`?

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01),
              metrics=["accuracy"])

log = model.fit(X_train, y_train,
                epochs=30, batch_size=256,
                validation_data=(X_valid, y_valid))

Notice the number of steps taken in each epoch (the counter just underneath "Epoch x/30"). Can you explain where the number of steps are coming from? What do you notice about the time taken?

### Using Momentum

We can add momentum to many algorithms. The base case is to add momentum to `SGD`. A typical value is `0.9` but keep in mind that this is another hyperparameter that may need some tuning. When setting up `SGD`, you can also tick `nesterov=True` to use the Nesterov algorithm, a moment-based algorithm we didn't discuss. Try it out:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01, momentum=0.9, nesterov=False),
              metrics=["accuracy"])
log = model.fit(X_train, y_train,epochs=3,batch_size=128,
                validation_data=(X_valid, y_valid))
create_plot(log)

### RMSpop

RMSprop is an algorithm that pursues a slightly different idea: it normalizes the gradients using their squares. It requires to specify a `learning_rate`, as well as the hyperparameters `rho` and `epsilon`. For the latter two, the standard values usually do just fine, while even the `learning_rate` is less problematic than in `SGD`.

If you want, you can also add `momentum` to the algorithm.

Try it out:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-07, momentum=0.0),
              metrics=["accuracy"])
log = model.fit(X_train, y_train,epochs=3,batch_size=128,
                validation_data=(X_valid, y_valid))
create_plot(log)

### Adam

Finally, we have `Adam`, which is used as commonly (or even more) than `RMSprop`. Adam combines the ideas of RMSprop and momentum gradient descent. However, it also adds a slight adjustment that is particularly relevant for early iterations. The hyperparameters are `learning_rate`, `beta_1`, `beta_2`, and `epislon`, even though mostly people leave everything but the `learning_rate` alone. Try it out:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
              metrics=["accuracy"])
log = model.fit(X_train, y_train,epochs=3,batch_size=128,
                validation_data=(X_valid, y_valid))
create_plot(log)

### Specific initializations

We generally want to initialize our weights in a sensible manner (especially if we are not using batch normalization, for example, because of runtime concerns). Let's start with our baseline model:

In [ ]:
os.environ['PYTHONHASHSEED']=str(631)
random.seed(631)
np.random.seed(631)
tf.random.set_seed(631)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

weights, biases = model.layers[1].get_weights()
print("First layer: " + str(weights[0,0]))
weights, biases = model.layers[-1].get_weights()
print("Last layer: " + str(weights[0,0]))

Can you copy the model definition, but change the first layer to `kernel_initializer='he_normal'` and the output layer to `kernel_initializer='glorot_uniform'`?

What changes do you observe in the first layer, what changes in the last layer? Do they make sense?

In [ ]:
os.environ['PYTHONHASHSEED']=str(631)
random.seed(631)
np.random.seed(631)
tf.random.set_seed(631)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform")
])

weights, biases = model.layers[1].get_weights()
print("First layer: " + str(weights[0,0]))
weights, biases = model.layers[-1].get_weights()
print("Last layer: " + str(weights[0,0]))

# 3. Enabling and speeding up training

Below, we create a much deeper neural network. As you can see, not much is happening in terms of learning for now:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="sigmoid"),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01),
              metrics=["accuracy"])
log = model.fit(X_train, y_train, epochs=10,batch_size=128,
                    validation_data=(X_valid, y_valid))

## 3.1 Batch normalization

Batch normalization allows us to do normalization at all stages of the network. For each input that is normalized, we need 4 parameters:
1. One that determines how the input is scaled (trainable)
1. One that determines how the input is shifted (trainable)
1. One that keeps track of the average of that input (non-trainable - it is still being adjusted though!)
1. One that keeps track of the standard deviation of that input (non-trainable - it is still being adjusted though!)

### Option 1: After activation (before inputs are weighted)

We can simply add a `BatchNormalization` layer before each of our `Dense` layers:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="sigmoid"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.summary()

Can you verify the number of trainable and non-trainable parameters?

Let's now train the network again:

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01),
              metrics=["accuracy"])
log = model.fit(X_train, y_train, epochs=5,batch_size=128,
                    validation_data=(X_valid, y_valid))

We can train even a deep neural network much more easily!

### Option 2: Before activation (after inputs are weighted)

The reommendation by the authors of the original paper on batch normalization is to normalize the weighted sum that goes into the neurons. That is, we first combine the inputs (and add a bias), then we "normalize" that weighted sum, before running the activation function on it. To do so in TensorFlow, we have to split apart our hidden layers into the combination and the activaiton. We out the `BatchNormalization` in-between

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(30),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(30),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(30),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(30),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(30),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(30),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(10),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('softmax')
])
model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.01),
              metrics=["accuracy"])
log = model.fit(X_train, y_train, epochs=5, batch_size=128,
                    validation_data=(X_valid, y_valid))

In practice, the differences between the two options tend to be small. But if you are really struggling to get your network to learn, try it out like this!

## 3.2 Learning rate schedule

### Power scheduling

Remember that each epoch contains a number of steps ($\frac{n}{\text{mini-batch-size}}$ to be exact). If we want to express our decay schedule based on the number of epochs, we first have to make a bit of an adjustment. For example, say that we specify the min-batch-size to 128 and that we want to have reach the next "decay step" (i.e., 1/2, 1/3, 1/4, ...) after 5 epochs.

Can you define the correct `s`, which should be the number of steps (not epochs!) until we reach the next "decay step"?

In [ ]:
batch_size = 128
epochs_until_change = 5
steps_per_epoch = X_train.shape[0] / batch_size
s = epochs_until_change * steps_per_epoch

print("epochs_until_change:",epochs_until_change)
print("steps_per_epoch:",steps_per_epoch)
print("s:",s)

Once we have defined the right `s`, we can train the model by manually defining our optimizer, using the TensorFlow scheduling process.

Here, we use `InverseTimeDecay` which computes
```
current_learning_rate = initial_learning_rate / (1 + decay_rate * step / decay_steps)
```
Increasing the `decay_rate` is equivalent to decreasing the `decay_steps`. Since we have already tuned `decay_steps=s`, we can simply set the `decay_rate` to 1.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

learning_rate = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=0.01, decay_steps=s, decay_rate=1)
optimizer = tf.keras.optimizers.experimental.Adam(learning_rate = learning_rate)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])
log = model.fit(X_train, y_train,
                epochs=10,batch_size=batch_size,
                validation_data=(X_valid, y_valid))
create_plot(log)

Let's compare this to the case with just a flat `learning_rate`:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
optimizer = tf.keras.optimizers.experimental.Adam(learning_rate = 0.01)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])
log = model.fit(X_train, y_train,
                epochs=10,batch_size=batch_size,
                validation_data=(X_valid, y_valid))
create_plot(log)

### Exponential scheduling

We now use the `ExponentialDecay` schedule. Here, the computation is
```
current_learning_rate = initial_learning_rate * decay_rate**(step / decay_steps)
```
(Note that `**` means to the power of)

Our baseline exponential schedule has a `0.1` base, so we set `decay_rate=0.1`:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01, decay_steps=s, decay_rate=0.1)
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])
log = model.fit(X_train, y_train,
                epochs=10,batch_size=batch_size,
                validation_data=(X_valid, y_valid))
create_plot(log)

# 4. A lack of overfitting - increasing the capacity of the model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
log = model.fit(X_train, y_train,
                epochs=20,batch_size=128,
                validation_data=(X_valid, y_valid))
create_plot(log)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
log = model.fit(X_train, y_train,
                epochs=20,batch_size=128,
                validation_data=(X_valid, y_valid))
create_plot(log)

# 5. Reducing bias through hyperparameter tuning

One option to run HP tuning relies on TensorBoard and makes things quite visual. It's also relatively intuitive, but doesn't have as much functionality as the Keras Tuner, which we will see in the tutorial.

In [ ]:
from tensorboard.plugins.hparams import api as hp

In [ ]:
%load_ext tensorboard

The following command deletes the log folder and may be useful for cleaning up. But be careful not to delete the things you still want to keep:

In [ ]:
rm -rf ./logs/

We start by defining the parameters to tune over. We will tune the learning rate, the choice of optimizer, the dropout rate, and the number of hidden units per layer. We will stay fixed with two hidden layers, however.

Using HParams, we define the parameters, as well as the interval over which we may vary them:

In [ ]:
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,1.0))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'rmsprop']))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete(range(50,150)))

We also need to define the metrics to measure. We will only care about the accuracy in our case, since we are performing a classification on balanced data:

In [ ]:
METRIC_ACCURACY = 'accuracy'

Once we have set up our parameters and metrics, we write those into our folder with the logs:

In [ ]:
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(hparams=[HP_LEARNING_RATE, HP_OPTIMIZER, HP_DROPOUT, HP_NUM_UNITS],
                      metrics = [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')])

Next, we define a function that creates and trains a model, and evaluates it on the test set. This function will get a dictionary `hparams`, that contains the different parameter choices. Hence, the way the model is build is kept variable.

The function also logs the choice of parameters and the output of our function (the mse), in order to display both in TensorBoard. For this purpose, we give it the current directory where the relevant information should be kept, `run_dir`.

In [ ]:
def train_test_model(hparams, run_dir):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation="relu"),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")])
    
    if hparams[HP_OPTIMIZER] == 'rmsprop':
        optimizer = tf.keras.optimizers.experimental.RMSprop(learning_rate=hparams[HP_LEARNING_RATE])
    elif hparams[HP_OPTIMIZER] == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE])
        
    model.compile(  optimizer=optimizer,
                    loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=5)
    accuracy = model.evaluate(X_valid, y_valid)[1]
    
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

Finally, we run a few different choices of parameters. Remember to choose parameters randomly! It is fine to sample the `dropout_rate`, `num_units` and `optimizer` at uniformly, but keep in mind the scaling issue when it comes to the `learning_rate`:

In [ ]:
total_sessions = 3

for session in range(total_sessions):
    
    # Create hyperparameters randomly
    dropout_rate = HP_DROPOUT.domain.sample_uniform()
    num_units = HP_NUM_UNITS.domain.sample_uniform()
    optimizer = HP_OPTIMIZER.domain.sample_uniform()
    
    r = -3*np.random.rand()
    learning_rate = 10.0**r
    
    # Create a dictionary of hyperparameters
    hparams = { HP_LEARNING_RATE: learning_rate,
                HP_OPTIMIZER: optimizer,
                HP_DROPOUT: dropout_rate,
                HP_NUM_UNITS: num_units}
    
    # train the model with the chosen parameters
    run_name = "run-%d" % session
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    train_test_model(hparams,'logs/hparam_tuning/' + run_name)

Finally, we display the runs using TensorBoard. If you are lucky, it is enough to call
```
%tensorboard --logdir logs
```
Otherwise, you might have to specify the path to the TensorBoard binary, such as here:

In [ ]:
os.environ['TENSORBOARD_BINARY'] = '/Users/philippe/anaconda3/envs/adl_env/bin/tensorboard'
%tensorboard --logdir logs

Once you have chosen a model that you like based on the hyperparameter search, how do you proceed?

# 6. Variance

## 6.1 Spurious correlations as a source of variance

We will see here that noise in your data makes your predictions worse - because it introduces spurious correlations that the model "learns" but that are actually without any meaning for generalization (i.e., that lead to more and more overfitting).

To show this, let's add additional features to the data. In one case, we will add features that are all zero, and in another case we will add features that are random. Note that we flatten the data first in this case, to make this process easier:

In [ ]:
X_flat = X_other.reshape(X_other.shape[0],X_other.shape[1]**2)
X_with_noise = np.concatenate([X_flat, np.random.random((X_flat.shape[0],X_flat.shape[1]))], axis=1)
X_with_zero = np.concatenate([X_flat, np.zeros((X_flat.shape[0],X_flat.shape[1]))],axis=1)

Take a look at the shapes of our new data sets:

In [ ]:
print(X_flat.shape)
print(X_with_noise.shape)
print(X_with_zero.shape)

We will now create the same model twice. Note that we no longer need a `Flatten` layer, because we already flattened the data. Also, we only applied our transformation once, to the `X_other` data. Hence, instead of resplitting our data, we will let `TensorFlow` do it for us, using `validation_split`.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
log_noise = model.fit(X_with_noise, y_other,
                        epochs=5,batch_size=128,
                        validation_split = 0.2)
log_zero = model.fit(X_with_zero, y_other,
                        epochs=5,batch_size=128,
                        validation_split = 0.2)

Plot the results below. You will see that adding noise is always detrimental to the generalization ability of your model. The more noise, the worse!

In [ ]:
plt.plot(log_noise.history['val_accuracy'],label = "Noise added",color='green')
plt.plot(log_zero.history['val_accuracy'], label = "Zero added",color='darkblue')
plt.title("Validation accuracies")
plt.legend()
plt.show()

# 6.2 Lowering variance through regularization

We learned about a number of regularization techniques. Here, we will see how to implement early stopping, L2-regularization, and dropout-regularization

### Early stopping

When we implement early stopping, the model definition and compilation is unchanged:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

However, we now need to add a `callback` to the training process. We define the `EarlyStopping` callback, which interrupts training if the validation loss is no longer improving. In particular, the callback waits for `patience` epochs of no improvement before interrupting:

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

The other parameter here is `restore_best_weights`. If set to `True`, this simply means that, once the callback decides to interrupt, it takes the version of the model that led to the best validation loss so far (do you know which epoch this corresponds to?)

In [ ]:
log = model.fit(X_train, y_train, epochs=50,batch_size=128,
                validation_data=(X_valid, y_valid),
                callbacks=[early_stopping_cb])
create_plot(log)

### L2- regularization

L2-regularization adds a penalty based on the L2-norm to the loss function. Usually, we add the same penalty for all weights, and we don't add a penalty for the biases. But you could also add a `bias_regularizer`, or even an `activity_regularizer`, which regularizes the output of the neurons instead of the parameters.

Keep in mind that the regularization parameter is another hyperparameter that might need tuning. A good starting point is 0.01, but it can vary quite a bit depending on the problem and network.

In [ ]:
reg_param = 0.01
regularizer = tf.keras.regularizers.l2(reg_param)

Can you rerun the model from above, but using regularization? In particular, to each `Dense` layer, you want to add the argument `kernel_regularizer=regularizer`:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(512, activation="relu",kernel_regularizer=regularizer),
    tf.keras.layers.Dense(10, activation="softmax",kernel_regularizer=regularizer)
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
log = model.fit(X_train, y_train, epochs=10,batch_size=128,
                    validation_data=(X_valid, y_valid))
create_plot(log)

As the regularization term is added when training but not when validating/testing, the loss will typically start out quite high, before the optimization routine finds a good way to adjust the weights to reduce the loss.

Note that we have improved the overfitting issue quite a bit, but unfortunately we have made it more difficult for the model to learn (we introduced bias). This is frequently the case, and we usually need to do some fiddling to find a good compromise.

### Dropout-regularization

Another regularization method is dropout-regularization. At training time, in each iteration, a number of neurons will be considered as non-existent, so we force the network to distribute weights more equally across neurons.

This makes the correct computation for activations a bit challenging when doing predictions, but luckily TensorFlow takes care of the added complexity.

If we want to ensure that neurons at a certain layer drop out (with probability `rate`), we add a `Dropout` layer before the corresponding `Dense` layer, using
```
tf.keras.layers.Dropout(rate=0.2)
```
Of course, `0.2` is just a particular choice and we can vary that.
Can you repeat the previous (baseline) model, but adding a `Dropout` layer before each `Dense` layer?

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
log = model.fit(X_train, y_train, epochs=10,batch_size=128,
                    validation_data=(X_valid, y_valid))
create_plot(log)